In [ ]:
import os
import cv2
import json
path = "./data/"

file = os.listdir(path)
# print(type(file)) <class 'list'>
# print(file) 
'''
['create_result_gif.py', 'demo.ipynb', 'webcam_demo.py', 
'demo.jpg', '00001.json', 'video_demo.py', 'demo.sh', 
...
'webcam_demo.py'
]
'''

jpg_file = []
json_file = []
for label in file:
    if label[-3:] == "jpg":
        jpg_file.append(label)
    else :
        json_file.append(label)


# print(len(jpg_file),jpg_file)
# print(len(json_file),json_file)

#读取json
def readJson(jsonfile):
    with open(jsonfile,encoding='utf-8') as f:
        jsonData = json.load(f) # json.load():将已编码的 JSON 字符串解码为 Python 对象
    return jsonData

srcimg = []
jsondata = []

for i in range(len(jpg_file)):
    srcimg.append(cv2.imread(path + jpg_file[i]))
    jsondata.append(readJson(path + json_file[i]))
    # print(type(srcimg),type(jsondata))
    jsondata[i].append(str(jpg_file[i]))

# print(type(jsondata),len(jsondata))


#     2 ['00001.jpg', '00002.jpg']
#     2 ['00001.json', '00002.json']
#     <class 'list'> 12 
#     {'color': [100, 255, 0, 100],
#     'height': 0.04437621310353279, 
#     'label': 'out1', 
#     'points': [[0.10383327305316925, 0.22243930399417877]], 
#     'type': 'Rectangle', 
#     'width': 0.0703386664390564}


In [1]:
"""本模块用于批量转换labelme标记的格式，使之变成coco数据集格式"""
# -*- coding:utf-8 -*-
# !/usr/bin/env python


import argparse
import json
import matplotlib.pyplot as plt
import skimage.io as io
import cv2
from labelme import utils
import numpy as np
import glob
import PIL.Image


class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)


class labelme2coco(object):
    def __init__(self, labelme_json=[], save_json_path='./tran.json'):
        '''
        labelme_json: 所有labelme的json文件路径组成的列表
        save_json_path: json保存位置
        '''
        self.labelme_json = labelme_json
        self.save_json_path = save_json_path
        self.images = []
        self.categories = []
        self.annotations = []
        # self.data_coco = {}
        self.label = []
        self.annID = 1
        self.height = 0
        self.width = 0
        self.root = ''

        self.save_json()  # 这里就是labelme2coco类的一个构造函数，
                          # 只要创建类的同时就会按照规则初始化类的成员值以及这个调用函数
                          # 而这个函数的定义在后面
                          # 所以简单来说这里一调用该类就直接运行相关的代码了,牛逼


    def data_transfer(self):
        i=-1
        for num, json_file in enumerate(self.labelme_json): # labelme_json是一个列表，也就是对应路径下的所有*.json文件的文件名的list
            print("num:" + str(num + 1) + '    ' + json_file)

            with open(json_file, 'r') as fp:
                data = json.load(fp)  # 加载json文件,data通常是一个list类型，具体内容根据标注json文件内容而定
                ## 注意这里要学会根据customize_datasets去筛选需要转化的信息项目中sockets中的json文件里面我们最需要关注的是每个框选范围内物体的label
                for per_data in data: 
                    i = i+1

                    ## 注意这里append里面的image是一个函数，函数的定义在后面功能是将对应图片的信息转化成coco的image格式
                    self.images.append(self.image(per_data,i,str(json_file)))  
                    label = per_data['label'] # per_data是一个dict,根据key值索引到label
                    if label not in self.label:
                        self.categories.append(self.categorie(label)) 
                        # self.categorie也是一个函数,函数定义在后面,功能是将生成coco的categories格式 

                        self.label.append(label)
                    points = per_data['points']  # 这里的point是用rectangle标注得到的，只有两个点，需要转成四个点
                    #points.append([points[0][0], points[1][1]])
                    #points.append([points[1][0], points[0][1]])

                    # 这里给出左上角坐标和标注框的height和width,需要加上框的其他点
                    points[0][0] = points[0][0] * self.width 
                    points[0][1] = points[0][1] * self.height
                    points.append([ points[0][0],points[0][1]+per_data['height']*self.height ])
                    points.append([ points[0][0]+per_data['width']*self.width,points[0][1]+per_data['height']*self.height ])
                    points.append([ points[0][0]+per_data['width']*self.width,points[0][1] ])
                    # 这里也是self.annotation也是一个函数,功能是将记录的所有points信息和segmentation文件记录下来
                    # points.pop(0)
                    self.annotations.append(
                        self.annotation(points, label, i) )
                    self.annID += 1

    def image(self, data, id,name):
        image = {}
        # img = utils.img_b64_to_arr(data['imageData'])  # 解析原图片数据 但是sockets标注中没有data['imageData']所以不能通过这种格式转换要采用下面的格式
        # img = cv2.imread(data['imagePath'], 0)
        img=io.imread(os.path.join(self.root,name[:-4]+'jpg')) # 通过图片路径打开图片
        height,width = img.shape[:2]  # 这里因为标注信息里面没有存储imagedata相关的信息,所以需要读取文件获得
        # 这里的height,width也是整张图片的height,width
        img = None
        image['width'] = width
        image['height'] = height
        image['id'] = id
        #image['file_name'] = data['imagePath'].split("\\")[-1] #此行不适合我的数据集
        # image['file_name'] = data['imagePath'].split("\\")[-1].split('..')[-1]
        image['file_name'] = name[-10:-4] + 'jpg'

        self.height = height
        self.width = width

        return image

    def categorie(self, label):
        categorie = {}
        categorie['supercategory'] = 'Cancer' #这个对我们这个项目来说没啥用,但存起来也无妨
        categorie['id'] = len(self.label) + 1  # 0 默认为背景
        categorie['name'] = label
        return categorie

    def annotation(self, points, label, id):
        annotation = {}
        annotation['segmentation'] = [list(np.asarray(points).flatten())] # 由于本项目sockets标注记录的信息只有框的一个点和height和width
                                                                          # 转化后将其记录进segmentation即可
        annotation['iscrowd'] = 0
        annotation['image_id'] = id 
        # annotation['bbox'] = str(self.getbbox(points)) # 使用list保存json文件时报错（不知道为什么）
        # list(map(int,a[1:-1].split(','))) a=annotation['bbox'] 使用该方式转成list

        annotation['bbox'] = list(map(float, self.getbbox(points))) # 在我们标注的数据集中不适合这样取转化bbox信息
 
        annotation['area'] = annotation['bbox'][2] * annotation['bbox'][3]
        # annotation['category_id'] = self.getcatid(label)
        annotation['category_id'] = self.getcatid(label)  # 注意，源代码默认为1
        annotation['id'] = self.annID

        print(annotation['bbox'],'\n\n')  # 每次转化结束都看一下bbox信息是否出现错误
        return annotation

    def getcatid(self, label):
        for categorie in self.categories:
            if label == categorie['name']:
                return categorie['id']
        return 1

    def getbbox(self, points):
        # img = np.zeros([self.height,self.width],np.uint8)
        # cv2.polylines(img, [np.asarray(points)], True, 1, lineType=cv2.LINE_AA)  # 画边界线
        # cv2.fillPoly(img, [np.asarray(points)], 1)  # 画多边形 内部像素值为1
        polygons = points

        mask = self.polygons_to_mask([self.height, self.width], polygons)
        #print(polygons)
        return self.mask2box(mask)

    def mask2box(self, mask):
        '''从mask反算出其边框
        mask：\[h,w]  0、1组成的图片
        1对应对象，只需计算1对应的行列号（左上角行列号，右下角行列号，就可以算出其边框）
        '''
        # np.where(mask==1)
        index = np.argwhere(mask == 1)
        #print(index)


        rows = index[:, 0]
        clos = index[:, 1]
        # 解析左上角行列号
        # print(rows,'\n',clos)
        # if(rows.size==0 or clos.size==0):
        #     return [0,0,0,0]

        
        left_top_r = np.min(rows)  # y 行号
        left_top_c = np.min(clos)  # x 列号


        # 解析右下角行列号
        right_bottom_r = np.max(rows)
        right_bottom_c = np.max(clos)

        # return [(left_top_r,left_top_c),(right_bottom_r,right_bottom_c)]
        # return [(left_top_c, left_top_r), (right_bottom_c, right_bottom_r)]
        # print([(left_top_c, left_top_r), (right_bottom_c, right_bottom_r)])  # [x1,y1,x2,y2]
        # print(left_top_r.shape,left_top_r,"hah")
        # print(left_top_c.shape,left_top_c,"hah")
        return [left_top_c, left_top_r, right_bottom_c - left_top_c,
                right_bottom_r - left_top_r]  # [x1,y1,w,h] 对应COCO的bbox格式

    def polygons_to_mask(self, img_shape, polygons):
        print(polygons)
        mask = np.zeros(img_shape,dtype=np.uint8)
        mask = PIL.Image.fromarray(mask)
        print(type(mask))
        xy = list(map(tuple, polygons))
        PIL.ImageDraw.Draw(mask).polygon(xy=xy, outline=1, fill=1)
        mask = np.array(mask, dtype=bool)
        # print(type(mask),mask.shape,'\n',mask)
        return mask

    def data2coco(self):
        data_coco = {}
        data_coco['images'] = self.images
        data_coco['categories'] = self.categories
        data_coco['annotations'] = self.annotations
        return data_coco

    def save_json(self):
        self.data_transfer()
        self.data_coco = self.data2coco()
        # 保存json文件
        json.dump(self.data_coco, open(self.save_json_path, 'w'), indent=4, cls=MyEncoder)  # indent=4 更加美观显示


# labelme_json = glob.glob('/home/gyf/data/sockets/*.json')
# print(type(labelme_json),len(labelme_json),labelme_json,type(labelme_json[0]))
# <class 'list'> 2 ['/home/gyf/data/sockets/00001.json', '/home/gyf/data/sockets/00002.json']

# labelme_json=['./Annotations/*.json']
# labelme_json=glob.glob('./Annotations/*.json')

# saveme_json = '/home/gyf/projects/mmdetection/gyf/draft/draft.json'
# print("Start.")

# labelme2coco(labelme_json, saveme_json)
# print("Finished.")

# coco_train:
print("coco_train:")
train_json = glob.glob("/home/gyf/projects/mmdetection/data/sockets/train/*.json")
print(train_json)
saveme_json = "/home/gyf/projects/mmdetection/data/sockets/train/annotation_coco.json"
print("Start.")
labelme2coco(train_json,saveme_json)
print("Finished.")

# coco_val:
print("coc_val:")
val_json = glob.glob("/home/gyf/projects/mmdetection/data/sockets/val/*.json")
print(val_json)
saveme_json = "/home/gyf/projects/mmdetection/data/sockets/val/annotation_coco.json"
print("Start.")
labelme2coco(val_json,saveme_json)
print("Finished.")

coco_train:
['/home/gyf/projects/mmdetection/data/sockets/train/00139.json', '/home/gyf/projects/mmdetection/data/sockets/train/00117.json', '/home/gyf/projects/mmdetection/data/sockets/train/00027.json', '/home/gyf/projects/mmdetection/data/sockets/train/00015.json', '/home/gyf/projects/mmdetection/data/sockets/train/00014.json', '/home/gyf/projects/mmdetection/data/sockets/train/00086.json', '/home/gyf/projects/mmdetection/data/sockets/train/00018.json', '/home/gyf/projects/mmdetection/data/sockets/train/00109.json', '/home/gyf/projects/mmdetection/data/sockets/train/00148.json', '/home/gyf/projects/mmdetection/data/sockets/train/00176.json', '/home/gyf/projects/mmdetection/data/sockets/train/00114.json', '/home/gyf/projects/mmdetection/data/sockets/train/00096.json', '/home/gyf/projects/mmdetection/data/sockets/train/00120.json', '/home/gyf/projects/mmdetection/data/sockets/train/00153.json', '/home/gyf/projects/mmdetection/data/sockets/train/00160.json', '/home/gyf/projects/mmdetec